In [1]:
#### In this notebook I obtain the branching ratio for the processes zp -> chi chi, 
#### y0 -> chi chi, and h2 -> chi chi, and store the results in a dataframe that
#### contains the mediator mass, and the branching ratios

import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import scipy
import matplotlib as mpl
import sympy as sp
from matplotlib.colors import LogNorm



plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

In [2]:
BRdata = pd.DataFrame(columns=['Mediator', 'mass', '$g_{q}$', '$\sin\\theta$', 
                               '$g_{\chi}$', '$y_{chi}$', '$BR(med>\chi\chi)$'])

In [3]:
## spin 1 mediator ##

# mediator mass
MZp = np.arange(140, 3010, 10)

# quarks masses
MU = 0.00255
MC = 1.27
MT = 172
MD = 0.00504
MS = 0.101
MB = 4.7

# DM mass
Mchi = 65.0

# couplings
gq = np.array([0.01, 0.1, 0.2, 0.25])
gchi = np.array([0.1, 0.5, 1.0, 2.0])
gqA = 0.0
gZp = 1.0


In [4]:
for g2 in gq:
    for g1 in gchi:
        for m in MZp:
            zpcc = ((-48*gqA**2*MC**2 + 24*g2**2*gZp**2*MC**2 + 12*gqA**2*m**2 
                     + 12*g2**2*gZp**2*m**2)*np.sqrt(-4*MC**2*m**2 + m**4))/(48*np.pi*abs(m)**3)
            if m < 2*MT:
                zptt = 0
            else:
                zptt = ((-48*gqA**2*MT**2 + 24*g2**2*gZp**2*MT**2 + 12*gqA**2*m**2 
                         + 12*g2**2*gZp**2*m**2)*np.sqrt(-4*MT**2*m**2 + m**4))/(48*np.pi*abs(m)**3)
            zpdd = ((-48*gqA**2*MD**2 + 24*g2**2*gZp**2*MD**2 + 12*gqA**2*m**2 
                     + 12*g2**2*gZp**2*m**2)*np.sqrt(-4*MD**2*m**2 + m**4))/(48*np.pi*abs(m)**3)
            zpss = ((-48*gqA**2*MS**2 + 24*g2**2*gZp**2*MS**2 + 12*gqA**2*m**2 
                     + 12*g2**2*gZp**2*m**2)*np.sqrt(-4*MS**2*m**2 + m**4))/(48*np.pi*abs(m)**3)
            zpbb = ((-48*gqA**2*MB**2 + 24*g2**2*gZp**2*MB**2 + 12*gqA**2*m**2 
                     + 12*g2**2*gZp**2*m**2)*np.sqrt(-4*MB**2*m**2 + m**4))/(48*np.pi*abs(m)**3)
            zpchichi = ((-16*g1**2*Mchi**2 + 4*g1**2*m**2)*np.sqrt(-4*Mchi**2*m**2 
                                                                + m**4))/(96*np.pi*abs(m)**3)

            BRZpChi = zpchichi/(zpcc + zptt + zpdd + zpss + zpbb + zpchichi)
                
#             d = {'Mediator': '$Z^{\prime}$', 'mass': m, '$g_{q}$': g2, '$g_{\chi}$': g1, 
#                  '$BR(med>\chi\chi)$': BRZpChi}
#             df = pd.DataFrame(d)
            
            BRdata.loc[len(BRdata)] = ['$Z^{\prime}$', m, g2, np.nan, g1, np.nan, BRZpChi]

In [5]:
BRdata

,Mediator,mass,$g_{q}$,$\sin\theta$,$g_{\chi}$,$y_{chi}$,$BR(med>\chi\chi)$
0,$Z^{\prime}$,140,0.01,NaN,0.1,NaN,0.175622
1,$Z^{\prime}$,150,0.01,NaN,0.1,NaN,0.340963
2,$Z^{\prime}$,160,0.01,NaN,0.1,NaN,0.452200
3,$Z^{\prime}$,170,0.01,NaN,0.1,NaN,0.527152
4,$Z^{\prime}$,180,0.01,NaN,0.1,NaN,0.579602
...,...,...,...,...,...,...,...
4587,$Z^{\prime}$,2960,0.25,NaN,2.0,NaN,0.680224
4588,$Z^{\prime}$,2970,0.25,NaN,2.0,NaN,0.680229
4589,$Z^{\prime}$,2980,0.25,NaN,2.0,NaN,0.680233
4590,$Z^{\prime}$,2990,0.25,NaN,2.0,NaN,0.680237


In [9]:
## spin-0 mediator ##

# mediator mass
MSd = np.arange(140, 810, 10)

# quarks masses
MU = 0.00255
MC = 1.27
MT = 172
MD = 0.00504
MS = 0.101
MB = 4.7

# DM mass
Mchi = 65.0

# gauge bosons masses
MW = 80.4
MZ = 91.2
MZp = 3000.0

# higgs boson mass
MH1 = 125.3

# vevs
vev = 246
gchi = 1.0
xev = MZp/(2*gchi)

# couplings
ychi = np.array([0.1, 0.5, 1.0, 2.0])
yc = np.sqrt(2)*MC/xev
yt = np.sqrt(2)*MT/xev
ydo = np.sqrt(2)*MD/xev
ys = np.sqrt(2)*MS/xev
yup = np.sqrt(2)*MU/xev
yb = np.sqrt(2)*MB/xev
cw = MW/MZ
sw = np.sqrt(1 - cw**2)
ee = (2*MW*sw)/vev
Sa = np.array([0.1, 0.2])


In [10]:
for s in Sa:
    for y in ychi:
        for m in MSd:
            Ca = np.sqrt(1-s**2)
            lam1 = (MH1**2 + m**2 + (MH1**2 - m**2)*(Ca**2 - s**2))/(4*vev**2)
            lam2 = (gchi**2*(MH1**2 + m**2 + (-MH1**2 + m**2)*(Ca**2 - s**2)))/MZp**2
            lam3 = (2*Ca*gchi*(MH1**2 - m**2)*s)/(MZp*vev)
            
            if m < 2*MH1:
                sdhh = 0
            else:
                sdhh = ((36*Ca**4*lam1**2*s**2*vev**2 - 24*Ca**4*lam1*lam3*s**2*vev**2 + 4*Ca**4*lam3**2*s**2*vev**2 
                         + 12*Ca**2*lam1*lam3*s**4*vev**2 - 4*Ca**2*lam3**2*s**4*vev**2 + lam3**2*s**6*vev**2 
                         + 12*Ca**5*lam1*lam3*s*vev*xev - 4*Ca**5*lam3**2*s*vev*xev + 72*Ca**3*lam1*lam2*s**3*vev*xev 
                         - 24*Ca**3*lam1*lam3*s**3*vev*xev - 24*Ca**3*lam2*lam3*s**3*vev*xev + 10*Ca**3*lam3**2*s**3*vev*xev 
                         + 12*Ca*lam2*lam3*s**5*vev*xev - 4*Ca*lam3**2*s**5*vev*xev + Ca**6*lam3**2*xev**2 
                         + 12*Ca**4*lam2*lam3*s**2*xev**2 - 4*Ca**4*lam3**2*s**2*xev**2 + 36*Ca**2*lam2**2*s**4*xev**2 
                         - 24*Ca**2*lam2*lam3*s**4*xev**2 
                         + 4*Ca**2*lam3**2*s**4*xev**2)*np.sqrt(-4*MH1**2*m**2 + m**4))/(32*np.pi*abs(m)**3)
            if m < 2*MT:
                sdtt = 0
            else:
                sdtt = (3*(m**2 - 4*MT**2)*np.sqrt(m**4*(1 - (4*MT**2)/m**2))*s**2*yt**2)/(16*np.pi*(m)**3)
                
            if m < 2*MW:
                sdww = 0
            else:
                sdww = (ee**4*np.sqrt(m**4*(1 - (4*MW**2)/m**2))*(12*MW**4 + m**4*(1 - (4*MW**2)/m**2))*s**2*vev**2)/(256*MW**4*np.pi*sw**4*(m)**3)

            if m < 2*MZ:
                sdzz = 0
            else:
                sdzz = -1/32*(MW**4*np.sqrt(m**4*(1 - (4*MZ**2)/m**2))*(-12*MZ**4 + m**4*(-1 + (4*MZ**2)/m**2))*s**2*(cw**2 + sw**2)**4)/(cw**4*MZ**4*np.pi*vev**2*(m)**3)
            sdcc = (3*np.sqrt((1 - (4*MC**2)/m**2)*m**4)*(-4*MC**2 + m**2)*s**2*yc**2)/(16*np.pi*(m)**3)
            sddd = (3*np.sqrt((1 - (4*MD**2)/m**2)*m**4)*(-4*MD**2 + m**2)*s**2*ydo**2)/(16*np.pi*(m)**3)
            sdss = (3*(m**2 - 4*MS**2)*np.sqrt(m**4*(1 - (4*MS**2)/m**2))*s**2*ys**2)/(16*np.pi*(m)**3)
            sdbb = (3*np.sqrt((1 - (4*MB**2)/m**2)*m**4)*(-4*MB**2 + m**2)*s**2*yb**2)/(16*np.pi*(m)**3)
            sduu = (3*(m**2 - 4*MU**2)*np.sqrt(m**4*(1 -(4*MU**2)/m**2))*s**2*yup**2)/(16*np.pi*(m)**3)
            sdchichi = (Ca**2*np.sqrt((1 - (4*Mchi**2)/m**2)*m**4)*(-4*Mchi**2 + m**2)*y**2)/(32*np.pi*(m)**3)
            
            BRSdChi = sdchichi / (sdhh + sdww + sdzz + sduu + sdcc + sdtt + sddd + sdss + sdbb + sdchichi)
            
            BRdata.loc[len(BRdata)] = ['$S_{d}$', m, np.nan, s, np.nan, y, BRSdChi]

In [14]:
BRdata

,Mediator,mass,$g_{q}$,$\sin\theta$,$g_{\chi}$,$y_{chi}$,$BR(med>\chi\chi)$
0,$Z^{\prime}$,140,0.01,NaN,0.1,NaN,0.175622
1,$Z^{\prime}$,150,0.01,NaN,0.1,NaN,0.340963
2,$Z^{\prime}$,160,0.01,NaN,0.1,NaN,0.452200
3,$Z^{\prime}$,170,0.01,NaN,0.1,NaN,0.527152
4,$Z^{\prime}$,180,0.01,NaN,0.1,NaN,0.579602
...,...,...,...,...,...,...,...
5651,$S_{d}$,760,NaN,0.2,NaN,2.0,0.761271
5652,$S_{d}$,770,NaN,0.2,NaN,2.0,0.756232
5653,$S_{d}$,780,NaN,0.2,NaN,2.0,0.751194
5654,$S_{d}$,790,NaN,0.2,NaN,2.0,0.746158


In [15]:
BRdata.to_pickle('../BRdata.pcl')